In [142]:
import pandas as pd
from mol2vec import features
from rdkit import Chem
from gensim.models import word2vec
import numpy as np
import pandas as pd
import sys, os
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import IPythonConsole

from mol2vec.features import mol2alt_sentence, mol2sentence, MolSentence, DfVec, sentences2vec
from gensim.models import word2vec

In [143]:
data = pd.read_csv("../data/data.csv", index_col="id")

In [144]:
data = data.reset_index(drop=True)

In [145]:
from rdkit import Chem

data['Molecules'] = data['smiles'].apply(Chem.MolFromSmiles)


In [148]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import pandas as pd

# Предполагается, что у вас уже есть DataFrame `data` с колонкой 'SMILES'
# data = pd.read_csv('path_to_your_data.csv')

# Пример расчета базовых дескрипторов (повторение для контекста)
def calculate_basic_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return {desc[0]: None for desc in Descriptors.descList}
    
    descriptors = {}
    for desc_name, desc_func in Descriptors.descList:
        try:
            descriptors[desc_name] = desc_func(mol)
        except Exception:
            descriptors[desc_name] = None
    return descriptors

# Функция для добавления Morgan Fingerprints
def add_morgan_fingerprints(smiles, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
        return list(map(int, fp))
    else:
        return [None] * n_bits

# Добавляем базовые дескрипторы
basic_descriptors_list = data['smiles'].apply(calculate_basic_descriptors).tolist()
basic_descriptors_df = pd.DataFrame(basic_descriptors_list)

# Добавляем Morgan Fingerprints
morgan_fp_df = pd.DataFrame(data['smiles'].apply(lambda x: add_morgan_fingerprints(x, 1024)).tolist())

# Объединяем DataFrame'ы
combined_data = pd.concat([data, basic_descriptors_df, morgan_fp_df], axis=1)


In [118]:
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd

# Предполагается, что данные уже загружены в DataFrame `data` с колонкой SMILES
# data = pd.read_csv('path_to_your_data.csv')

# Функция для расчета всех доступных дескрипторов RDKit для молекулы
def calculate_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:  # Если молекула не может быть создана
        return {desc[0]: None for desc in Descriptors.descList}
    
    descriptors = {}
    for desc_name, desc_func in Descriptors.descList:
        try:
            descriptors[desc_name] = desc_func(mol)
        except Exception as e:
            descriptors[desc_name] = None
    return descriptors

# Применение функции к каждой SMILES в датасете и создание нового DataFrame с результатами
descriptors_list = data['smiles'].apply(calculate_all_descriptors).tolist()
descriptors_df = pd.DataFrame(descriptors_list)

# Если требуется, можно объединить исходный DataFrame с новым DataFrame дескрипторов
combined_data = pd.concat([data, descriptors_df], axis=1)

# Теперь `combined_data` содержит исходные данные вместе с расширенным набором дескрипторов


In [149]:
combined_data = combined_data.drop(columns=["smiles", "Molecules"])

In [163]:
from sklearn.model_selection import train_test_split

features = combined_data.drop(columns=["lgK"])  # И другие дескрипторы, которые вы решили использовать
targets = combined_data['lgK']

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


In [164]:
features

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,9.770833,-1.583333,9.770833,1.201389,0.485716,104.105,96.041,104.047344,42,0,...,0,0,0,0,0,0,0,0,0,0
1,10.233333,-1.500000,10.233333,0.262731,0.591504,132.159,120.063,132.078644,54,0,...,0,0,0,0,0,0,0,0,0,0
2,10.273333,-1.388889,10.273333,0.418981,0.538992,130.143,120.063,130.062994,52,0,...,0,0,0,0,0,0,0,0,0,0
3,10.426111,-1.404514,10.426111,0.418981,0.571016,144.170,132.074,144.078644,58,0,...,0,0,0,0,0,0,0,0,0,0
4,10.249726,-1.406296,10.249726,0.402778,0.658691,152.149,144.085,152.047344,58,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,10.353148,-1.167870,10.353148,0.079444,0.531152,167.120,162.080,167.021858,62,0,...,0,0,0,0,0,0,0,0,0,0
243,9.680324,-1.180556,9.680324,0.273148,0.511508,104.105,96.041,104.047344,42,0,...,0,0,0,0,0,0,0,0,0,0
244,9.839213,-1.166667,9.839213,0.342593,0.557411,118.132,108.052,118.062994,48,0,...,0,0,0,0,0,0,0,0,0,0
245,9.890741,-1.212963,9.890741,0.201389,0.539805,118.132,108.052,118.062994,48,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_seed=42,
    thread_count=-1,
    eval_metric="RMSE",
    verbose=500,)
model.fit(X_train, y_train)

# Предсказания
predictions = model.predict(X_test)


Learning rate set to 0.031674
0:	learn: 6.2292085	total: 1.68ms	remaining: 1.67s
500:	learn: 0.4039151	total: 958ms	remaining: 954ms
999:	learn: 0.1213894	total: 1.97s	remaining: 0us


In [166]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, predictions)

1.3322516584898414

In [167]:
pd.DataFrame({"feature": features.columns.values, "importance": model.feature_importances_})

,feature,importance
0,MaxEStateIndex,0.190301
1,MinEStateIndex,0.837809
2,MaxAbsEStateIndex,0.419662
3,MinAbsEStateIndex,1.443679
4,qed,2.167202
...,...,...
1227,1019,0.000000
1228,1020,0.000000
1229,1021,0.000000
1230,1022,0.000000


In [1]:
model.get_feature_importance(prettified=True)

NameError: name 'model' is not defined